In [15]:
import pandas as pd
from data_tweak.converters import convert_datetime, convert_race, convert_sex

stops_df = pd.read_csv("../files/events/stops_full.csv")
stops_df.dropna(subset=["date_time"], how="any", inplace=True)
convert_datetime(stops_df)
# this needs to be sorted so we can slice properly
stops_df.set_index("date_time", inplace=True)
stops_df.sort_index(inplace=True)
convert_race(stops_df, ["first_off_race", "second_off_race", "civ_race"])
convert_sex(stops_df, ["first_off_sex", "second_off_sex", "civ_sex"])


/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_58378/4243400398.py:4: DtypeWarning: Columns (3,5,6,7,8,10,11,12,13,20,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  stops_df = pd.read_csv("../files/events/stops_full.csv")


In [29]:

import numpy as np


"""
  @TODO bring in second officers on stops
  @TODO bring in arrest/uof data
  @TODO be more explicit about data we're excluding; record it
  @TODO identify potential mis-merges
"""

with pd.read_csv("../files/events/assignment_cleaned_first_200k.csv", chunksize=10000) as reader:
  left_on = ["off_last_name", "off_first_name", "off_race", "off_sex"]
  right_on = ["first_off_last_name", "first_off_first_name", "first_off_race", "first_off_sex"]
  for chunk in reader:
    chunk.dropna(subset=["shift_start", "shift_end_corrected"], how="any", inplace=True)
    convert_datetime(chunk, datetime_col="shift_start")
    convert_datetime(chunk, datetime_col="shift_end_corrected")
    convert_race(chunk, ["off_race"])
    convert_sex(chunk, ["off_sex"])
    chunk.set_index("shift_start", inplace=True, drop=False)
    window_start = chunk["shift_start"].min()
    window_end = chunk["shift_end_corrected"].max()
    """ right now, this is only assignment records for which there is a stop """
    stops_merged = pd.merge(chunk, stops_df[window_start:window_end], how="left", left_on=left_on, right_on=right_on).dropna(subset=right_on)
    stops_merged = stops_merged.assign(
      stops_white = np.where(stops_merged["civ_race"] == "WHITE", 1, 0),
      stops_black = np.where(stops_merged["civ_race"] == "BLACK", 1, 0),
      stops_hispanic = np.where(stops_merged["civ_race"] == "HISPANIC", 1, 0),
      stops_apa = np.where(stops_merged["civ_race"] == "ASIAN/PACIFIC ISLANDER", 1, 0),
      stops_native = np.where(stops_merged["civ_race"] == "NATIVE AMERICAN/ALASKAN NATIVE", 1, 0),
      stops_male = np.where(stops_merged["civ_sex"] == "MALE", 1, 0),
      stops_female = np.where(stops_merged["civ_sex"] == "FEMALE", 1, 0)
    )
    grped = stops_merged.groupby(["shift_start", "shift_end_corrected", "off_first_name", "off_last_name", "off_sex", "off_race"]).agg({"stops_white":sum, "stops_black":sum, "stops_hispanic":sum, "stops_apa":sum, "stops_native":sum, "stops_male":sum, "stops_female":sum,})
    grped.to_csv("../files/events/assignment_merged/merge_grp_test.csv", mode="a")
    # stops_merged.to_csv("../files/events/assignment_merged/merge_test.csv")
    # pd.merge(chunk, stops_df[window_start:window_end], how="left", left_on=left_on, right_on=right_on).dropna(subset=right_on).to_csv("../files/events/assignment_merged/merge_test.csv", mode="a")


/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_58378/3233935613.py:16: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  stops_merged = pd.merge(chunk, stops_df[window_start:window_end], how="left", left_on=left_on, right_on=right_on).dropna(subset=right_on)
/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_58378/3233935613.py:16: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  stops_merged = pd.merge(chunk, stops_df[window_start:window_end], how="left", left_on=left_on, right_on=right_on).dropna(subset=right_on)
/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_58378/3233935613.py:16: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and wil